In [2]:
import pandas as pd
from sklearn.metrics import (
    roc_curve,
    auc,
    precision_recall_curve,
    average_precision_score,
)
import numpy as np
import plotly.express as px

In [3]:
df = pd.read_csv(
    "/results/rr/study/hg38s/study252-P200_analysis/workflow_results/chr15q14/analysis_overview_golga8a_unphased.tsv", sep="\t"
)
df["CT_length"] = df["length"] * df["%CT"]
df["aFTLD-U"] = df["group"].apply(lambda x: 1 if x == "aFTLD-U" else 0)
hap_encoding = {"major": 1, "minor": 1, "none": 0}
df["haplotype_encoded"] = df["haplotype"].apply(lambda x: hap_encoding[x])
df.drop_duplicates(subset=["name"], inplace=True)

samples = pd.read_csv("/home/wdecoster/chr15q14/full_cohort_for_paper.tsv", sep="\t")
samples = samples[
    (samples["inclusion"] == "yes") & (samples["collection"].isin(["normal", "1000G"]))
]
df = df[df["name"].isin(samples["individual"])]

FileNotFoundError: [Errno 2] No such file or directory: '/results/rr/study/hg38s/study252-P200_analysis/workflow_results/chr15q14/analysis_overview_golga8a_unphased.tsv'

In [ ]:
# make a strip plot of the CT_dimer_count and aFTLD-U status

fig = px.strip(df, x="CT_dimer_count", y="group", color="haplotype", title="CT dimer count vs aFTLD-U status", hover_data=["name"], labels={"CT_dimer_count": "CT dimer count", "aFTLD-U": "aFTLD-U status"})
fig.show()

NameError: name 'px' is not defined

In [3]:
def roc(df, variable, title="ROC curve"):
    fpr, tpr, thresholds = roc_curve(df["aFTLD-U"], df[variable])
    f1_scores = 2 * tpr * (1 - fpr) / (tpr + 1 - fpr)
    roc_auc = auc(fpr, tpr)

    import plotly.graph_objects as go

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=fpr,
            y=tpr,
            mode="lines",
            name=f"ROC curve (area = {roc_auc:.2f})",
            line=dict(color="darkorange", width=2),
        )
    )
    # also show the diagonal
    fig.add_trace(
        go.Scatter(
            x=[0, 1],
            y=[0, 1],
            mode="lines",
            name="Random",
            line=dict(color="navy", width=2, dash="dash"),
            showlegend=False,
        )
    )

    # limit the x and y axis to 0-1
    fig.update_xaxes(
        range=[0, 1], mirror=True, showline=True, linecolor="black", linewidth=2
    )
    fig.update_yaxes(
        range=[0, 1], mirror=True, showline=True, linecolor="black", linewidth=2
    )

    fig.update_layout(
        title=title,
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        showlegend=True,
        plot_bgcolor="white",
        font=dict(size=20),
        width=600,
        height=600,
        legend=dict(
            yanchor="bottom",
            y=0,
            xanchor="right",
            x=1,
            bordercolor="black",
            borderwidth=1,
        ),
    )

    fig.show()

In [25]:
# plot a ROC curve for classifying individuals based on increasing lengths of the CT_dimer_count column, compared to the aFTLD-U column


roc(df, variable="CT_dimer_count", title="ROC for CT_dimer_count<br>for all individuals")

In [26]:
# roc curve, but for the haplotype
roc(df, variable="haplotype_encoded", title="ROC for haplotype-tagging variant for all individuals")

In [5]:
# now do the same, but only for major haplotype carriers
roc(df[df["haplotype"] == "major"], "CT_dimer_count", title="ROC for CT_dimer_count<br>for haplotype A carriers")


In [6]:
# and the minor haplotype carriers

roc(df[df["haplotype"] == "minor"], "CT_dimer_count", title="ROC for CT_dimer_count<br>for haplotype B carriers")

In [8]:
roc(df, "CT_length", title="ROC for CT_length<br>for all individuals")

In [9]:
# same for major haplotype carriers
roc(df[df["haplotype"] == "major"], "CT_length", title="ROC for CT_length<br>for haplotype A carriers")

In [10]:
# and the minor haplotype carriers
roc(df[df["haplotype"] == "minor"], "CT_length", title="ROC for CT_length<br>for haplotype B carriers")

In [11]:
roc(df, "%CT", title="ROC for %CT<br>for all individuals")

In [12]:
roc(df[df["haplotype"] == "major"], "%CT", title="ROC for %CT<br>for haplotype A carriers")

In [13]:
roc(df[df["haplotype"] == "minor"], "%CT", title="ROC for %CT<br>for haplotype B carriers")

In [14]:
# do the same plots but just for length
roc(df, "length", title="ROC for length<br>for all individuals")

In [15]:
roc(df[df["haplotype"] == "major"], "length", title="ROC for length<br>for haplotype A carriers")

In [16]:
roc(df[df["haplotype"] == "minor"], "length", title="ROC for length<br>for haplotype B carriers")

In [83]:
def precision_recall(df, variables, title="Precision recall curve"):
    import plotly.graph_objects as go

    fig = go.Figure()

    for variable in variables:
        precision, recall, thresholds = precision_recall_curve(df["aFTLD-U"], df[variable], drop_intermediate=True)
        f1_scores = 2 * precision * recall / (precision + recall)
        print('Best threshold: ', thresholds[np.argmax(f1_scores)])
        print('Best F1-Score: ', np.max(f1_scores))

        pr_auc = average_precision_score(df["aFTLD-U"], df[variable])

        fig.add_trace(
            go.Scatter(
                x=recall,
                y=precision,
                mode="lines",
                name=f"PR curve for {variable} (AveP = {pr_auc:.2f})",
                line=dict(width=2),
            )
        )

    # limit the x and y axis to 0-1
    fig.update_xaxes(
        range=[0, 1], mirror=True, showline=True, linecolor="black", linewidth=2
    )
    fig.update_yaxes(
        range=[0, 1], mirror=True, showline=True, linecolor="black", linewidth=2
    )

    fig.update_layout(
        title=title,
        xaxis_title="Recall",
        yaxis_title="Precision",
        showlegend=True,
        plot_bgcolor="white",
        font=dict(size=20),
        width=600,
        height=600,
        legend=dict(
            yanchor="bottom",
            y=-0.4,
            xanchor="right",
            x=1,
            bordercolor="black",
            borderwidth=1,
        ),

    )

    fig.show()

In [84]:
precision_recall(df, variables=["haplotype_encoded", "CT_dimer_count"], title="Precision recall for CT_dimer_count<br>for all individuals")

Best threshold:  1
Best F1-Score:  0.5210084033613445
Best threshold:  196
Best F1-Score:  0.6744186046511628


In [60]:
precision_recall(
    df,
    variable="haplotype_encoded",
    title="Precision recall for haplotype<br>for all individuals",
)

Best threshold:  1
Best F1-Score:  0.5210084033613445


In [19]:
precision_recall(df[df["haplotype"] == "major"], "CT_dimer_count", title="Precision recall for CT_dimer_count<br>for haplotype A carriers")

Best threshold:  257
Best F1-Score:  0.8800000000000001


In [20]:
precision_recall(df[df["haplotype"] == "minor"], "CT_dimer_count", title="Precision recall for CT_dimer_count<br>for haplotype B carriers")

Best threshold:  204
Best F1-Score:  0.8750000000000001


In [21]:
precision_recall(
    df[df["haplotype"].isin(["minor", "major"])],
    "CT_dimer_count",
    title="Precision recall for CT_dimer_count<br>for haplotype A+B carriers",
)

Best threshold:  196
Best F1-Score:  0.8695652173913043


In [44]:
# same but for CT_length
precision_recall(df, "CT_length", title="Precision recall for CT_length<br>for all individuals")

Best threshold:  435.699
Best F1-Score:  0.6170212765957447


In [45]:
precision_recall(df[df["haplotype"] == "major"], "CT_length", title="Precision recall for CT_length<br>for haplotype A carriers")

Best threshold:  557.844
Best F1-Score:  0.84


In [46]:
precision_recall(df[df["haplotype"] == "minor"], "CT_length", title="Precision recall for CT_length<br>for haplotype B carriers")

Best threshold:  450.576
Best F1-Score:  0.823529411764706


In [25]:
# now I would like to predict the aFTLD-U status by combinding the %CT and the repeat length in a logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


def log_reg(df):
    X = df[["%CT", "length"]]
    y = df["aFTLD-U"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    model = LogisticRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print(classification_report(y_test, y_pred, target_names=["%CT", "length"]))
    # how do I use the weighted average of the two columns in a roc curve?
    
    


In [26]:
log_reg(df)

              precision    recall  f1-score   support

         %CT       0.97      1.00      0.98       471
      length       0.80      0.21      0.33        19

    accuracy                           0.97       490
   macro avg       0.88      0.60      0.66       490
weighted avg       0.96      0.97      0.96       490



In [27]:
# for the major haplotype carriers
log_reg(df[df["haplotype"] == "major"])

              precision    recall  f1-score   support

         %CT       0.50      0.50      0.50         6
      length       0.62      0.62      0.62         8

    accuracy                           0.57        14
   macro avg       0.56      0.56      0.56        14
weighted avg       0.57      0.57      0.57        14



In [28]:
# for the minor haplotype carriers
log_reg(df[df["haplotype"] == "minor"])

              precision    recall  f1-score   support

         %CT       1.00      1.00      1.00         8
      length       1.00      1.00      1.00         3

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11



In [29]:
# make a strip lot of the CT_dimer_count, colored by aFTLD-U status
import plotly.express as px

hap_dict = {"major": "Haplotype A", "minor": "Haplotype B"}
df["Haplotype"] = df["haplotype"].apply(lambda x: hap_dict.get(x, "None"))
df["Phenotype"] = df["aFTLD-U"].apply(lambda x: "aFTLD-U" if x == 1 else "Control")

fig = px.strip(df, x="CT_dimer_count", y="Haplotype", color="Phenotype", title="CT_dimer_count vs aFTLD-U status", hover_data=["name"], 
               labels={"CT_dimer_count": "CT-dimer [count]"})
fig.add_vline(x=190, line_dash="dash", line_color="black")

fig.update_layout(
    plot_bgcolor="white",
    font=dict(size=20),
    width=1200,
    height=400,
)
fig.update_xaxes(
    showline=True,
    linewidth=2,
    linecolor="black",
    mirror=True,
)

fig.update_yaxes(
    showline=True,
    linewidth=2,
    linecolor="black",
    mirror=True,
)

fig.show()